In [34]:
import pandas as pd
import numpy as np
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import precision_at_k
from sklearn.model_selection import ParameterGrid, KFold

In [35]:
# importamos el archivo
df_inicial = pd.read_csv('0_products_df.csv')

In [36]:
# eliminamos columna
df_inicial.drop('Unnamed: 0', axis=1, inplace= True)

In [37]:
# eliminamos filas nan
df_inicial = df_inicial.dropna(how='any')

In [38]:
# creamos una lista con los productos
products = df_inicial.columns[2:]

In [39]:
# creamos un user_item anotando el numero de observaciones por producto
user_item_df = df_inicial.groupby('pk_cid').sum()

In [40]:
# vamos a quitar aquellos usuarios que no han contratado nunca ningun servicio
user_item_df['suma'] = user_item_df.sum(axis=1)
user_item_df = user_item_df[user_item_df['suma']!=0]

# eliminamos la columna de filtrado
user_item_df.drop('suma',axis=1, inplace=True)

In [41]:
user_item_df

,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
pk_cid,,,,,,,,,,,,,,,
15891,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
16203,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,5
16502,0,0,0,0,0,0,0,0,0.0,0.0,0,5,0,0,8
17457,0,0,0,0,0,16,0,0,0.0,0.0,0,0,5,0,17
17799,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553456,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1553541,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1553559,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [42]:
user_item_df.sum()

short_term_deposit      15394.0
loans                     468.0
mortgage                  324.0
funds                   20099.0
securities              22133.0
long_term_deposit      100660.0
em_account_pp               0.0
credit_card             70866.0
payroll                205787.0
pension_plan           217802.0
payroll_account        329420.0
emc_account            326961.0
debit_card             562966.0
em_account_p               34.0
em_acount             4381601.0
dtype: float64

In [43]:
user_item_df.drop(['em_account_pp','em_account_p'], axis=1,inplace=True)

In [44]:
# creamos el item_user matrix
item_user_df = user_item_df.T

In [45]:
# con respecto a la valoracion por observaciones, queremos que las contrataciones repetidas tengan mayor confianza, 
# pero que este efecto disminuya a medida que aumenta el numero de contrataciones repetidas para reducir el impacto
# que un fanatico del producto pueda tener. Tambien queremos desviar parte del peso de la confirnza de los productos populares. 
# Para ello utilizaremos bm25.
item_user_df = bm25_weight(item_user_df, K1=100, B=0.8)

In [46]:
user_item_df_weighted = item_user_df.T.tocsr()

In [47]:
# Vamos a realizar una busqueda de hiperparametros con validacion cruzada. 
# Al no existir esta opcion en implicit, debemos de crearla nosotros.

# Definimos la cuadrícula de parámetros
param_grid = {
    'factors': [50, 100, 200],
    'regularization': [0.01, 0.1, 1.0],
    'iterations': [10, 20, 30]
}

# Inicializamos variables para almacenar los mejores resultados
mejores_parametros = None
mejor_precision_promedio = 0.0

# Realizamos la búsqueda en cuadrícula con validación cruzada (KFold)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for params in ParameterGrid(param_grid):
    precision_promedio = 0.0

    for train_index, test_index in kf.split(user_item_df_weighted):
        modelo = AlternatingLeastSquares(**params)
        modelo.fit(user_item_df_weighted[train_index])

        # Evaluamos los modelos en el conjunto de prueba
        precision = precision_at_k(modelo, user_item_df_weighted[train_index], user_item_df_weighted[test_index], K=5)
        precision_promedio += precision

    precision_promedio /= kf.get_n_splits()

    # Actualiza los mejores resultados si se encuentra una mejora
    if precision_promedio > mejor_precision_promedio:
        mejor_precision_promedio = precision_promedio
        mejores_parametros = params

print("Mejores parámetros:", mejores_parametros)
print("Mejor precisión promedio en la validación cruzada:", mejor_precision_promedio)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/70077 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/70077 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/70077 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/70076 [00:00<?, ?it/s]

Mejores parámetros: {'factors': 200, 'iterations': 10, 'regularization': 0.01}
Mejor precisión promedio en la validación cruzada: 0.3213191796064317


In [48]:
# entrenamos el modelo con los hiperparametros que mejor resultado nos han dado
model = AlternatingLeastSquares(factors=200, regularization=0.01, iterations = 10)
model.fit(user_item_df_weighted)

  0%|          | 0/10 [00:00<?, ?it/s]

In [49]:
# Creamos un DF con el producto que recomendaremos a cada uno de los clientes y su rating

# Listas temporales para almacenar resultados
usuario_list = []
scores_list = []
ids_list = []

for userid in np.arange(len(user_item_df.index)):
    ids, scores = model.recommend(userid, user_item_df_weighted[userid], N=1, filter_already_liked_items=True)
    usuario_list.append(userid)
    scores_list.append(scores[0])
    ids_list.append(ids[0])

# Crear DataFrame a partir de listas
df_recomendacion = pd.DataFrame({'Usuarios':usuario_list, 'Scores': scores_list, 'IDs': ids_list})


In [52]:
# añadimos el pk_cid
df_recomendacion = df_recomendacion.merge(pd.DataFrame(data=user_item_df.index).reset_index().rename(columns={'index':'Usuarios'}))

In [53]:
# cambiamos los IDs por los nombres de los productos
dict_items = dict(zip(np.arange(len(list(user_item_df.columns))),user_item_df.columns))
df_recomendacion.IDs = df_recomendacion.IDs.replace(dict_items)

In [54]:
df_recomendacion.drop('Usuarios',axis=1,inplace=True)

In [55]:
# visualizamos
df_recomendacion.sort_values('Scores',ascending=False)

,Scores,IDs,pk_cid
89324,0.407977,payroll,1139164
117570,0.399964,payroll,1177654
1341,0.399963,payroll,414364
157805,0.399963,payroll,1238763
88459,0.399963,payroll,1137500
...,...,...,...
323160,0.004707,payroll,1499514
349477,0.004707,payroll,1549763
257018,0.004707,payroll,1403340
343332,0.004707,payroll,1536132
